<a href="https://colab.research.google.com/github/AeroPablete/MIAR03-Algoritmos-de-optimizacion-2023-2024/blob/main/Trabajo_Practico__Pablo_Onate_Villaverde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico

Nombre y Apellidos: Pablo Javier Oñate Villaverde<br>
Url: https://github.com/AeroPablete/MIAR03-Algoritmos-de-optimizacion-2023-2024/blob/main/Trabajo_Practico__Pablo_Onate_Villaverde.ipynb<br>
Google Colab: https://colab.research.google.com/drive/1QysLaz9hjPdtAjGqwAzGddz7Z4bG0csk?usp=sharing <br>
Problema:
>2. Organizar los horarios de partidos de La Liga<br>

`Descripción del problema:`)     

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que **maximice la
audiencia.**

Los horarios disponibles se conocen a priori y son los siguientes:

<table>
    <tbody>
        <tr>
            <td>Viernes</td>
            <td>20</td>
        </tr>
        <tr>
            <td>Sábado</td>
            <td>12, 16, 18, 20</td>
        </tr>
        <tr>
            <td>Domingo</td>
            <td>12, 16, 18, 20</td>
        </tr>
        <tr>
            <td>Lunes</td>
            <td>20</td>
        </tr>
    </tbody>
</table>

En primer lugar se clasifican los equipos en tres categorías según el numero de
seguidores( que tiene relación directa con la audiencia). Hay 3 equipos en la
categoría A, 11 equipos de categoría B y 6 equipos de categoría C

Se conoce estadísticamente la audiencia que genera cada partido según los equipos
que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

|             | Categoría A | Categoría B  | Categoría C   |
|-------------|-------------|--------------|---------------|
| Categoría A | 2 Millones  | 1,3 Millones | 1 Millones    |
| Categoría B |             | 0,9 Millones | 0,75 Millones |
| Categoría C |             |              | 0,47 Millones |

Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce
según los coeficientes de la siguiente tabl

Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes

|     | Viernes | Sábado | Domingo | Lunes |
|-----|---------|--------|---------|-------|
| 12h | -       | 0,55   | 0,45    | -     |
| 16h | -       | 0,7    | 0,75    | -     |
| 18h | -       | 0,8    | 0,85    | -     |
| 20h | 0,4     | 1      | 1       | 0,4   |a

Es posible la coincidencia de horarios pero en este
caso la audiencia de cada partido se verá afectada y
se estima que se reduce en porcentaje según la
siguiente tabla dependiendo del número de
coincidenc

| Coincidencias | -%  |
|---------------|-----|
| 0             | 0%  |
| 1             | 25% |
| 2             | 45% |
| 3             | 60% |
| 4             | 70% |
| 5             | 75% |
| 6             | 78% |
| 7             | 80% |
| 8             | 80% |ias:

## Modelo

*Nota Bene*: Para comodidad del lector se ha agrupado todo el código en el apartado `Diseño`. Dicho código contiene numerosos comentarios para ayudar a la comprensión del mismo, razón por la cual se referenciará el mismo en numerosas ocasiones.

- **¿Cómo represento el espacio de soluciones?**

El objetivo de este problema es diseñar un algoritmo que construya la jornada de liga, que es una *colección* de partidos. En cualquier lenguaje de programación, y en especial en Python, la forma más sencilla de representar este tipo de colecciones es por una *lista*.

Por otro lado, para tener perfectamente definido un partido necesitamos los datos siguientes:
- los dos equipos que se enfrentan
- el horario del mismo

Representaremos un partido (en Python) como un *diccionario* con los campos o atributos siguientes:
- un atributo *teams* que es una tupla de números enteros. Cada uno de esos números es una referencia o clave del diccionario definido en el apartado `Diseño >> Carga de los datos del problema >> TEAMS_DB`.
- Un atributo *date* en el que se codifica mediante una determinada cadena de caracteres el horario del partido. Dicha codificación se detalla a continuación:

| Código | Horario del partido |
|--------|---------------------|
| V20    | Viernes a las 20H   |
| S12    | Sábado a las 12H    |
| S16    | Sábado a las 16H    |
| S18    | Sábado a las 18H    |
| S20    | Sábado a las 20H    |
| D12    | Domingo a las 12H   |
| D16    | Domingo a las 16H   |
| D18    | Domingo a las 18H   |
| D20    | Domingo a las 20H   |
| L20    | Lunes a las 20H     |

En definitiva, un ejemplo de solución al problema es la siguiente

```python
solucion = [
    {'teams': (1,2), 'date': 'V20'},
    {'teams': (3,4), 'date': 'S12'},
    {'teams': (5,6), 'date': 'S16'},
    {'teams': (7,8), 'date': 'S18'},
    {'teams': (9,10), 'date': 'S20'},
    {'teams': (11,12), 'date': 'D12'},
    {'teams': (13,14), 'date': 'D16'},
    {'teams': (15,16), 'date': 'D18'},
    {'teams': (17,18), 'date': 'D20'},
    {'teams': (19,20), 'date': 'L20'}
]
```

- **¿Cúal es la función objetivo?**

La función objetivo se encuentra debidamente codificada y comentada en el apartado `Diseño >> Función objetivo >> compute_match_week_audience`

El objetivo del problema es encontrar la jornada de partidos que *maxiza* la audiencia estimada. La audiencia de una jornada se define de la manera siguiente:

$$
audiencia_{jornada}=\sum_{i=1}^{10}audiencia_{partido-i}
$$

En el cálculo de la audiencia de un partido entran en juego tres variables:
- `Diseño >> Carga de los datos del problema >> BASE_AUDIENCE_DB`, una audiencia 'base' que depende de la categoría de los equipos y que resulta ser una estimación de la audiencia que tendría el partido de disputarse un sábado a las 20H (franja de máxima audiencia)
- `Diseño >> Carga de los datos del problema >> MATCH_DATE_PENALTY_DB`, una penalización (en %) debido al hecho de que el partido puede no haber sido planificado en las franjas de máxima audiencia (sábado o domingo a las 20H)
- `Diseño >> Carga de los datos del problema >> SAME_DATE_PENALTY_DB`, una penalización (en %) debido al hecho de que varios partidos pueden haber sido planificados en la misma franja horaria 'obligando' al espectador a elegir cual ver.

En definitiva, y respetando la nomenclatura definida previamente, la audiencia estimada para un partido puede calcularse como

$$
audiencia = base\_audience*match\_date\_penalty*(1-same\_date\_penalty)
$$

- **¿Cómo implemento las restricciones?**

Para facilitar la tarea, se recupera el ejemplo de candidata a solución óptima presentada anteriormente:

```python
solucion = [
    {'teams': (1,2), 'date': 'V20'},
    {'teams': (3,4), 'date': 'S12'},
    {'teams': (5,6), 'date': 'S16'},
    {'teams': (7,8), 'date': 'S18'},
    {'teams': (9,10), 'date': 'S20'},
    {'teams': (11,12), 'date': 'D12'},
    {'teams': (13,14), 'date': 'D16'},
    {'teams': (15,16), 'date': 'D18'},
    {'teams': (17,18), 'date': 'D20'},
    {'teams': (19,20), 'date': 'L20'}
]
```

- Restricción 1: Una jornada está compuesta por 10 partidos, es decir la variable *solución* ha de ser una lista que contenga exactamente 10 elementos.
- Restricción 2: Un partido lo juegan 2 equipos, es decir el atributo *teams* de cada uno de los diccionarios que representa un partido corresponde a una tupla de exactamente dos elementos.
- Restricción 3: Un equipo no puede disputar varios partidos en una misma jornada, es decir los *índices* (por ejemplo (2,3)) que representan los equipos no pueden estar repetidos en una mismo objeto solución.
- Restricción 4: Los partidos han de disputarse en las franjas horarias planteadas en el enunciado, es decir el valor del atributo *date* para cada uno de los partidos ha de ser uno de los siguientes {'V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20'}
- Restricción 5: Se debe asignar siempre un partido el viernes, es decir sólo un elemento de la lista *solución* puede tener su atributo *date* igual a 'V20'
- Restricción 6: Se debe asignar siempre un partido el lunes, es decir sólo un elemento de la lista solución puede tener su atributo date igual a 'L20'

En definitiva, el ejemplo de candidata a solución óptima presentada anteriormente es perfectamente válida.

## Análisis
- **¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**

Para proceder al conteo de las posibles soluciones del problema se procederá en dos etapas:
- En la primera, se busca determinar de cuántas formas se pueden generar los enfrentamientos entre los equipos, es decir una jornada.
- En la segunda, dada una jornada (es decir los 10 enfrentamientos ya definidos) se busca determinar de cuántas formas se pueden asignar horarios a dichos partidos.

#### Generación de una jornada

Se realizará el cálculo partido a partido.

Para el *primero* de ellos, dado que no se diferencia entre equipo local de equipo visitante, se trata de una combinación de 20 elementos tomados de dos en dos:

$$
C^{2}_{20}=\frac{20!}{2!(20-2)!} = \frac{20*19}{2}=190
$$

Para el *segundo* partido, se trata de una combinación de $20-2=18$ elementos tomados de nuevo de dos en dos:
$$
C^{2}_{18}=\frac{18!}{2!(18-2)!} = \frac{18*17}{2}=153
$$

Para el *tercer* partido, se trata de una combinación de $18-2=16$ elementos tomados de dos en dos:
$$
C^{2}_{16}=\frac{16!}{2!(16-2)!} = \frac{16*15}{2}=120
$$

Para el *cuarto* partido, se trata de una combinación de $16-2=14$ elementos tomados de dos en dos:
$$
C^{2}_{14}=\frac{14!}{2!(14-2)!} = \frac{14*13}{2}=91
$$

Para el *quinto* partido, se trata de una combinación de $14-2=12$ elementos tomados de dos en dos:
$$
C^{2}_{12}=\frac{12!}{2!(12-2)!} = \frac{12*11}{2}=66
$$

Para el *sexto* partido, se trata de una combinación de $12-2=10$ elementos tomados de dos en dos:
$$
C^{2}_{10}=\frac{10!}{2!(10-2)!} = \frac{10*9}{2}=45
$$

Para el *séptimo* partido, se trata de una combinación de $10-2=8$ elementos tomados de dos en dos:
$$
C^{2}_{8}=\frac{8!}{2!(8-2)!} = \frac{8*7}{2}=28
$$

Para el *octavo* partido, se trata de una combinación de $8-2=6$ elementos tomados de dos en dos:
$$
C^{2}_{6}=\frac{6!}{2!(6-2)!} = \frac{6*5}{2}=15
$$

Para el *noveno* partido, se trata de una combinación de $6-2=4$ elementos tomados de dos en dos:
$$
C^{2}_{4}=\frac{4!}{2!(4-2)!} = \frac{4*3}{2}=6
$$

Para el *décimo* partido, se trata de una combinación de $4-2=2$ elementos tomados de dos en dos:
$$
C^{2}_{2}=\frac{2!}{2!(2-2)!} = \frac{2*1}{2}=1
$$

Por lo tanto, el `número total de formas distintas de generar una jornada de 10 partidos` es $190*153*120*91*66*45*28*15*6*1=2.3758809*10^{15}$

#### Asignación de horarios

Dada una jornada (es decir los 10 enfrentamientos entre equipos), se busca determinar ahora de cuántas formas distintas es posible aisgnar un horario a cada uno de los partidos.

Considérese la siguiente tabla en la que cada casilla representa un *partido* de la jornada, y el número en su interior representa el *número de horarios* que podemos asignar a cada partido.

<table>
    <thead>
        <tr>
            <th>1 Partido</th>
            <th>2 Partido</th>
            <th>3 Partido</th>
            <th>4 Partido</th>
            <th>5 Partido</th>
            <th>6 Partido</th>
            <th>7 Partido</th>
            <th>8 Partido</th>
            <th>9 Partido</th>
            <th>10 partido</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: center">1</td>
            <td style="text-align: center">1</td>
            <td style="text-align: center">8</td>
            <td style="text-align: center">8</td>
            <td style="text-align: center">8</td>
            <td style="text-align: center">8</td>
            <td style="text-align: center">8</td>
            <td style="text-align: center">8</td>
            <td style="text-align: center">8</td>
            <td style="text-align: center">8</td>
        </tr>
    </tbody>
</table>

Fijados pues los partidos del viernes y del lunes (el primero y el segundo) para cumplir con las restricciones del problema, se dispone de 8 horarios para cada uno de los otros encuentros. Es decir, el número de asignaciones de horarios posibles en este caso es:
$$
1*1*8^{8}=16777216
$$

Por otro lado, determinar de cúantas formas es posible asignar los horarios del viernes y el lunes se reduce a una combinación de $10$ elementos tomados de dos en dos:
$$
C^{2}_{10}=\frac{10!}{2!(10-2)!} = \frac{10*9}{2}=45
$$

`La asignación de horarios (respetando las restricciones del problema) a una jornada ya definida` se puede hacer pues de $16777216*45=754974720$ formas diferentes.

#### Evaluación del número de soluciones del problema

De acuerdo con lo expuesto en los dos apartados anteriores, el número de soluciones posibles al problema, respetando las restricciones del mismo es del orden de $10^{23}$.

## Diseño
- **¿Que técnica utilizo? ¿Por qué?**

Dado el tamaño del espacio de búsqueda definido anteriormente, se decide utilizar una metaheurística como el *Recocido Simulado* (con multiarranque) para intentar encontrar una buena aproximación a la solución de este problema. La implementación de la misma se detalla a continuación.

#### Carga de las librerías

In [1]:
from collections import Counter
import math
import pprint as pp
import random
import time

#### Carga de los datos del problema

A continuación se procede a cargar los datos necesarios a la implementación del problema en forma de *diccionarios* en Python. Este tipo de estructuras *indexadas* permiten realizar búsquedas en ellas en tiempo $O(1)$

##### Equipos

Como se precisó en el apartado *Modelo*, los índices de este diccionario se utilizaran para referenciar los equipos cuando se construyan los partidos de una determinada jornada.

```python
match = {
    'teams': (1,2),
    'date': 'V20'
}
```

In [2]:
TEAMS_DB = {
    1: { 'category': 'CAT-A', 'name': 'Team 1' },
    2: { 'category': 'CAT-A', 'name': 'Team 2' },
    3: { 'category': 'CAT-A', 'name': 'Team 3' },
    4: { 'category': 'CAT-B', 'name': 'Team 4' },
    5: { 'category': 'CAT-B', 'name': 'Team 5' },
    6: { 'category': 'CAT-B', 'name': 'Team 6' },
    7: { 'category': 'CAT-B', 'name': 'Team 7' },
    8: { 'category': 'CAT-B', 'name': 'Team 8' },
    9: { 'category': 'CAT-B', 'name': 'Team 9' },
    10: { 'category': 'CAT-B', 'name': 'Team 10' },
    11: { 'category': 'CAT-B', 'name': 'Team 11' },
    12: { 'category': 'CAT-B', 'name': 'Team 12' },
    13: { 'category': 'CAT-B', 'name': 'Team 13' },
    14: { 'category': 'CAT-B', 'name': 'Team 14' },
    15: { 'category': 'CAT-C', 'name': 'Team 15' },
    16: { 'category': 'CAT-C', 'name': 'Team 16' },
    17: { 'category': 'CAT-C', 'name': 'Team 17' },
    18: { 'category': 'CAT-C', 'name': 'Team 18' },
    19: { 'category': 'CAT-C', 'name': 'Team 19' },
    20: { 'category': 'CAT-C', 'name': 'Team 20' }
}

##### Audiencias base de los partidos

En este caso los índices del diccionario están formados por tuplas que representan las categorías de los equipos.

In [3]:
BASE_AUDIENCE_DB = {
    ('CAT-A', 'CAT-A'): 2.0,
    ('CAT-B', 'CAT-B'): 0.9,
    ('CAT-C', 'CAT-C'): 0.47,
    ('CAT-A', 'CAT-B'): 1.3,
    ('CAT-B', 'CAT-A'): 1.3,
    ('CAT-A', 'CAT-C'): 1.0,
    ('CAT-C', 'CAT-A'): 1.0,
    ('CAT-B', 'CAT-C'): 0.75,
    ('CAT-C', 'CAT-B'): 0.75
}

##### Penalizaciones por horario

En este caso los índices del diccionario representan los horarios disponibles para los partidos y los valores, las 'penalizaciones' en términos de audiencia que se aplican a cada uno de los casos.

In [4]:
MATCH_DATE_PENALTY_DB = {
    'V20': 0.4,
    'S12': 0.55,
    'S16': 0.7,
    'S18': 0.8,
    'S20': 1,
    'D12': 0.45,
    'D16': 0.75,
    'D18': 0.85,
    'D20': 1,
    'L20': 0.4
}

##### Penalizaciones por horario repetido

En este caso los índices representan el número de ocasiones que se repite un mismo horario en una jornada, y los valores las 'penalizaciones' en términos de audiencia que se aplican en cada uno de los casos.

In [5]:
SAME_DATE_PENALTY_DB = {
    1: 0,
    2: 0.25,
    3: 0.45,
    4: 0.6,
    5: 0.7,
    6: 0.75,
    7: 0.78,
    8: 0.8
}

#### Función generadora de soluciones iniciales

En este apartado se detalla la función que permite generar soluciones aleatorias que se utilizarán como punto de partida del algoritmo.

In [6]:
def generate_match(teams_bag, dates_bag):
    """
    Función que permite generar un objeto de tipo partido
    a partir de un conjunto de equipos disponibles y otro
    de horarios disponibles.
    """
    # Selección de los equipos que disputarán el partido
    team_one = random.choice(list(teams_bag))
    team_two = random.choice(list(teams_bag))

    if team_one == team_two:
        return generate_match(teams_bag, dates_bag)

    # Selección del horario del partido
    match_date = random.choice(list(dates_bag))

    return {
        'teams': (team_one, team_two),
        'date': match_date
    }

In [7]:
def fix_match_week_dates(match_week):
    """
    Función que permite forzar que se respeten las restricciones del problema
    en el caso de que la solución inicial generada de forma aleatoria no las
    cumpla en su totalidad :
    - Un partido tiene que estar programado obligatoriamente el viernes a las 20H
    - Otro partido tiene que estar programado obligatoriamente el lunes a las 20H
    """
    # Si en la jornada generada de forma aleatoria no hay partido planificado el viernes,
    # lo forzamos
    friday_game = list(filter(lambda match: 'V20' == match['date'], match_week))
    if not friday_game:
        match_week[0]['date'] = 'V20'

    # Si en la jornada generada de forma aleatoria no hay partido planificado el lunes,
    # lo forzamos
    monday_game = list(filter(lambda match: 'L20' == match['date'], match_week))
    if not monday_game:
        match_week[9]['date'] = 'L20'

    return match_week

In [8]:
def generate_random_match_week():
    """
    Función que permite generar una jornada aleatoria que será el punto de partida
    del algoritmo de búsqueda
    """
    teams_bag = set(range(1, 21))
    dates_bag = {'V20', 'S12', 'S16', 'S18', 'S20', 'D12', 'D16', 'D18', 'D20', 'L20'}
    match_week = []

    for _ in range(10):
        match = generate_match(teams_bag, dates_bag)

        # Eliminamos la posibilidad de que se repitan equipos en dos partidos distintos
        teams_bag -= set(match['teams'])

        # Eliminamos la posibilidad de que haya dos partidos los viernes
        if 'V20' == match['date']:
            dates_bag -= {'V20'}

        # Eliminamos la posibilidad de que haya dos partidos los lunes
        if 'L20' == match['date']:
            dates_bag -= {'L20'}

        # Anadimos el partido a la jornada
        match_week.append(match)

    return fix_match_week_dates(match_week)

#### Función objetivo

En este apartado se detalla la función objetivo del problema que permite calcular la audiencia estimada para una jornada. En este caso se busca *maximizar* dicha función.

In [9]:
def compute_match_audience(match, stats):
    """
    Función que permite calcular la audiencia estimada para un partido
    de acuerdo a la formula siguiente:
    audiencia_estimada = audiencia_base * penalizacion_horario * penalizacion_repeticion_horario
    """
    teams = match['teams']
    date = match['date']

    # Determinemos la audiencia base del partido
    base_audience = BASE_AUDIENCE_DB[(TEAMS_DB[teams[0]]['category'], TEAMS_DB[teams[1]]['category'])]

    # Determinemos la penalizacion asociada al hecho de que el partido no se retransmite en
    # la franja de máxima audiencia
    match_date_penalty = MATCH_DATE_PENALTY_DB[date]

    # Determinemos la penalizacion asociada al hecho de que (quizas) varios partidos se hayan planificado
    # a la misma hora que el que nos ocupa
    same_date_penalty = 1 - SAME_DATE_PENALTY_DB[stats.get(date)]

    return base_audience * match_date_penalty * same_date_penalty

In [10]:
def compute_match_week_audience(match_week):
    """
    Función que permite calcular la audiencia estimada para una
    jornada completa planificada
    """
    # Contamos el numero de partidos que ocurren en la misma fecha
    # Nos servira para determinar una de las penalizaciones
    stats = Counter(match['date'] for match in match_week)

    # Calculamos la audiencia estimada para la jornada
    match_week_audience = 0
    match_week_audience_details = {}

    for index, match in enumerate(match_week):
        audience = compute_match_audience(match, stats)
        match_week_audience_details[index] = audience
        match_week_audience += audience

    return match_week_audience, match_week_audience_details

#### Función de vecindad

En este apartado se detalla la función que permite generar una solución vecina siguiendo los pasos descritos a continuación:
1. Se selecciona de forma aleatoria un conjunto de partidos ($m$) de la jornada y se extrae la lista de equipos y los horarios.
2. Se generan, de forma aleatoria, nuevos enfrentamientos entre esos $2m$ equipos
3. Se asigna, de forma aleatoria, un horario a esos nuevos $m$ enfrentamientos

Se intenta ahora aproximar el número de vecinos que es posible generar utilizando el algoritmo descrito previamente.

El número de nuevos enfrentamientos es:
$$
    C^{2}_{2m}C^{2}_{2m-2} \dots C^{2}_{2}=\frac{(2m)!}{2(2m-2)!}\frac{(2m-2)!}{2(2m-4)!}\frac{(2m-4)!}{2(2m-6)!} \dots \ \frac{2}{2.0!}=\frac{(2m)!}{2^{m}}
$$

Si, por simplicidad, se supone que no hay horarios repetidos en la lista extraida de la solución de partida, el número de asignaciones posibles es $m!$

El número de soluciones vecinas que es posible generar a partir de la de partida es pues de
$$
m!\frac{(2m)!}{2^{m}}
$$

En el código se toma como valor por defecto $m=4$ lo que equivale a $60480$ posibles soluciones, un vecindario que sigue siendo "pequeño" si se compara con el espacio de soluciones del problema.

In [11]:
def shuffle_match_list(match_list):
    """
    Función que permite mezclar los partidos de una lista
    (Esta puede corresponder a una jornada entera o a una porción de ella)
    """
    dates_bag = [match['date'] for match in match_list]
    teams_bag = [team for match in match_list for team in match['teams']]

    output = []
    for _ in range(len(match_list)):
        # Seleccionemos una fecha para el partido
        date = dates_bag.pop(random.randrange(len(dates_bag)))

        # Seleccionamos los equipos
        team_a = teams_bag.pop(random.randrange(len(teams_bag)))
        team_b = teams_bag.pop(random.randrange(len(teams_bag)))

        output.append({
            'date': date,
            'teams': (team_a, team_b)
        })

    return output

In [12]:
def get_indices_to_shuffle(match_week, shuffle_ratio=0.4):
    """
    Función que permite determinar, dada una jornada y un porcentaje de mezcla,
    los elementos a mezclar para obtener una nueva jornada vecina
    """
    indices = set()
    items = len(match_week)
    items_to_extract = math.floor(items * shuffle_ratio)

    while len(indices) < items_to_extract:
        indices.add(random.randrange(items))

    return list(indices)

In [13]:
def shuffle_neighbor_generator(match_week, shuffle_ratio=0.4):
    """
    Función que permite generar una jornada vecina
    a partir de una solución y dado un porcentaje de mezcla
    """
    # Separamos la lista de partidos en dos partes, una la mezclaremos
    # y la otra la dejaremos intacta
    indices = get_indices_to_shuffle(match_week, shuffle_ratio)
    hold_out_list = []
    to_shuffle_list = []

    for index, match in enumerate(match_week):
        if index in indices:
            to_shuffle_list.append(match)
        else:
            hold_out_list.append(match)

    # Retornamos la nueva jornada generada
    shuffled_list = shuffle_match_list(to_shuffle_list)
    return hold_out_list + shuffled_list

#### Función de búsqueda (Simulated Annealing)

In [14]:
def simulated_annealing_search(initial_solution, shuffle_ratio=0.4):
    """
    Algoritmo Simulated Annealing
    """
    # -- Guarda la solución inicial como la actual y la mejor hasta el momento --
    current_match_week = initial_solution
    current_audience, _ = compute_match_week_audience(initial_solution)
    best_match_week = initial_solution
    best_audience, _ = compute_match_week_audience(initial_solution)

    # -- Fijamos los parámetros del algoritmo --
    # Temperatura T
    temperature = current_audience * 0.3
    # Velocidad de enfriamiento alfa
    alpha = 0.9
    # El parámetro l determina el tiempo que decidimos quedarnos a un mismo nivel de temperatura
    l = 10000

    while temperature > 0.01:
        for _ in range(l):
            neighbor_match_week = shuffle_neighbor_generator(current_match_week, shuffle_ratio)
            neighbor_audience, _ = compute_match_week_audience(neighbor_match_week)

            # Si encontramos una solución de mejor calidad que la que teniamos guardada como "best"
            # nos la quedamos (obviamente)
            if neighbor_audience > best_audience:
                current_match_week = neighbor_match_week
                current_audience = neighbor_audience
                best_match_week = neighbor_match_week
                best_audience = neighbor_audience
                continue

            # Si la solución es de mejor calidad que la actual, nos la quedamos
            if neighbor_audience > current_audience:
                current_match_week = neighbor_match_week
                current_audience = neighbor_audience
                continue

            # Si la solución es de peor calidad que la actual, determinemos la probabilidad de quedarnosla
            delta = abs(neighbor_audience - current_audience)
            probability = math.exp(-1 * (delta / temperature))
            if random.random() < probability:
                current_match_week = neighbor_match_week
                current_audience = neighbor_audience
                continue

        # Descender la temperatura
        temperature *= alpha

    # Retornar la mejor solución encontrada
    return best_match_week, best_audience

#### Algoritmo de búsqueda (Simuleated annealing con multiarranque)

In [15]:
def search_algorithm(instances=10):
    """
    Algoritmo de búsqueda
    """
    audiences = []
    solutions = []
    for _ in range(instances):
        initial_solution = generate_random_match_week()
        match_week, audience = simulated_annealing_search(initial_solution, shuffle_ratio=0.4)
        solutions.append(match_week)
        audiences.append(audience)

    best_audience = max(audiences)
    best_match_week = solutions[audiences.index(best_audience)]
    return best_audience, best_match_week

In [16]:
# Lanzamos el algoritmo para ver como se comporta
# Para 10 ejecuciones tarda aproximadamente 2 minutos
tic = time.time()
best_audience, best_match_week = search_algorithm()
toc = time.time()

print(f"Mejor audiencia: {round(best_audience, 4)} (en {round(toc-tic, 2)} segundos)")
pp.pprint(best_match_week)

Mejor audiencia: 6.945 (en 179.26 segundos)
[{'date': 'L20', 'teams': (5, 17)},
 {'date': 'D12', 'teams': (19, 11)},
 {'date': 'S18', 'teams': (13, 16)},
 {'date': 'S16', 'teams': (7, 10)},
 {'date': 'S12', 'teams': (9, 20)},
 {'date': 'V20', 'teams': (15, 4)},
 {'date': 'D18', 'teams': (12, 6)},
 {'date': 'D20', 'teams': (8, 2)},
 {'date': 'S20', 'teams': (1, 3)},
 {'date': 'S18', 'teams': (14, 18)}]
